# Water level extraction

**A Jupyter notebook on how to detect the water level of water bodies around the world, using Sentinel-2 multi-spectral and multi-temporal imagery**

This notebook serves as an example on how to bring satellite data from space down to the hands of people living on Earth and analyze this data in order to make some conclusions that affect all of Earth's citizens. Specifically, it demonstrates how to run a water detection algorithm and extract surface water level for a single reservoir in a given time interval.

Hopefully, this example notebook promotes an increase of awareness about environmental problems and at least a bit helps us to make the world a better place.

## Notebook outline
The outline of this notebook is the following:
1. Defining geometries of a few waterbodies
2. Preparing and executing the full workflow for water detection
   1. Downloading Sentinel-2 data (true color and NDWI index) using [SentinelHub](https://www.sentinel-hub.com/) services
   2. Cloud detection using the [s2cloudless](https://github.com/sentinel-hub/sentinel2-cloud-detector) cloud detector
   3. Water detection
3. Visualizing the waterbodies and the water level over a period of time
4. Filtering out cloudy scenes to improve the results

## Requirements
- `eo-learn` (https://github.com/sentinel-hub/eo-learn)
- `Water Observatory Backend` (https://github.com/sentinel-hub/water-observatory-backend)

In order to run the example, you will also need a Sentinel Hub account. If you do not have one yet, you can create a free trial account at [Sentinel Hub webpage](https://services.sentinel-hub.com/oauth/subscription). If you are a researcher you can even apply for a free non-commercial account at [ESA OSEO page](https://earth.esa.int/aos/OSEO).

Once you have the account set up, login to [Sentinel Hub Configurator](https://apps.sentinel-hub.com/configurator/). By default, you will already have the default configuration with an **instance ID** (alpha-numeric code of length 36). For this tutorial, we recommend that you create a new configuration (`"Add new configuration"`) and set the configuration to be based on **Python scripts template**. Such configuration will already contain all layers used in these examples. Otherwise, you will have to define the layers for your configuration yourself.

After you have prepared a configuration please put configuration's **instance ID** into `sentinelhub` package's configuration file following the [configuration instructions](http://sentinelhub-py.readthedocs.io/en/latest/configure.html).

In [ ]:
# Set your Sentinel Hub instance ID either here or 
# in command line with sentinelhub.config --instance_id <your instance id>
INSTANCE_ID = None

### Define the necessary imports

In [ ]:
# set the autoreload and the inline plotting for matplotlib
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# download
import requests

# data manipulation
import numpy as np
import numpy.ma as ma

# geometry manipulation
import geopandas as gpd
import shapely.geometry
import shapely.wkt
from shapely.geometry import Point, MultiPolygon, Polygon

# image manipulation
import skimage.feature
import skimage.filters
import skimage.morphology
import rasterio

# Sentinel Hub
from sentinelhub import BBox, CRS

# eo-learn related
from eolearn.core import EOTask, LinearWorkflow, FeatureType
from eolearn.io import S2L1CWCSInput 
from eolearn.mask import AddCloudMaskTask, AddValidDataMaskTask, get_s2_pixel_cloud_detector
from eolearn.geometry import VectorToRaster

# plotting
import matplotlib
import matplotlib.patheffects
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## 1. Obtaining geometries of waterbodies

A waterbody chosen for this notebook:
- [Theewaterskloof Dam, South Africa](https://en.wikipedia.org/wiki/Theewaterskloof_Dam)

We can easily obtain waterbody geometries from the [BlueDot Water Observatory](https://water.blue-dot-observatory.com) API, which holds information of many waterbodies across the world. By searching for a specific waterbody, we can copy the `WATERBODY_ID` number in the URL in order to access the nominal geometry of the corresponding waterbody (i.e. number `38538` in URL `https://water.blue-dot-observatory.com/38538/2019-02-05`)

![Water Observatory screenshot](https://i.imgur.com/EqGI599.png)

In [ ]:
WATERBODY_ID = 38538

In [ ]:
def get_nominal_geometry(waterbody_id):
    """ A function for obtaining the nominal water geometry from the water observatory API
    """
    wb_url = f'https://water.blue-dot-observatory.com/api/waterbodies/{waterbody_id}/index.html'
    wb_data = requests.get(wb_url).json()
    
    return shapely.geometry.shape(wb_data['nominal_outline']['geometry'])


def plot_geometry(geom, ax=None, **kwargs):
    """ An utility function for plotting the geometry 
    """
    if geom is None or geom.exterior is None:
        return

    x,y = geom.exterior.xy
    if ax is None:
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(111)
        
    ax.plot(x, y, **kwargs)

In [ ]:
nominal_geometry = get_nominal_geometry(WATERBODY_ID)

plot_geometry(nominal_geometry)

But now we need a bounding box for this geometry, in order to download Sentinel-2 data. We define a bounding box and inflate it a little bit in order to construct a `BBox` object which is used with Sentinel Hub services. The BBox class also accepts the coordinate system (CRS), where we use the same one as in the case of the geometry, which is [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System).

In [ ]:
wb_bbox = BBox(nominal_geometry.bounds, crs=CRS.WGS84).buffer(0.2)

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
plot_geometry(wb_bbox.geometry, ax)
plot_geometry(nominal_geometry, ax)

## 2. Preparing and executing the full workflow for water detection

Sentinel Hub services are installed with `eo-learn`. It is an Earth observation processing framework for machine learning in Python, which provides seamless access and processing of spatio-temporal image sequences acquired by any satellite fleet in a timely and automatic manner.

`eo-learn` works as a workflow, where a workflow consists of one or multiple tasks. Each task achieves a specific job (downloading data, calculating band combinations, etc) on a small patch of an area, called EOPatch. EOPatch is a container for EO and non-EO data.

Let's define a workflow to download and obtain the necessary data for water detection. We will download the RGB bands in order to actually visualize the true-color image of the waterbody. Additionally, we will download the `NDWI` band combination, which we will use for water detection. It is defined as

$$NDWI = \frac{B_3-B_8}{B_3+B_8},$$

where $B_3$ and $B_8$ are the green and near-infrared Sentinel-2 bands, respectively.

### Definition of the main water detecition task

In [ ]:
class WaterDetectionTask(EOTask):
    """ A task for water mask and water level detection
    """
    def __init__(self, ndwi_feature, vector_result_feature, scalar_result_feature, nominal_geometry, *,
                 simplify=True, canny_sigma=4, canny_threshold=0.3, dilation_size=4):
        """
        :param ndwi_feature: Input feature of EOPatch containing NDWI values
        :type ndwi_feature: (FeatureType, str)
        :param vector_result_feature: A resulting vector feature containing waterbody geometries
        :type: vector_result_feature: (FeatureType, str)
        :param scalar_result_feature: A resulting scalar feature containing ratios between calculated area of detected
            waterbody and area of nominal waterbody
        :type scalar_result_feature: (FeatureType, str)
        :param nominal_geometry: A geometry of a waterbody
        :type nominal_geometry: shapely.geometry.Polygon or shapely.geometry.MultiPolygon
        :param simplify: A flag specifying if resulting geometry should be simplified
        :type simplify: bool
        :param canny_sigma: `sigma` parameter of `skimage.feature.canny` function
        :type canny_sigma: float
        :param canny_threshold: `high_threshold` parameter of `skimage.feature.canny` function
        :type canny_threshold: float
        :param dilation_size: Size of dilation in pixels used by `skimage.morphology.binary_dilation`
        :type dilation_size: int
        """
        self.ndwi_feature = ndwi_feature
        self.vector_result_feature = vector_result_feature
        self.scalar_result_feature = scalar_result_feature
        self.nominal_geometry = nominal_geometry

        self.simplify = simplify
        self.canny_sigma = canny_sigma
        self.canny_threshold = canny_threshold
        self.dilation_disk = skimage.morphology.disk(dilation_size)

    def _get_water_level(self, ndwi, bbox):
        """ Run water detection algorithm for an NDWI image
        """
        water_mask, water_status = self._get_water_mask_from_s2(ndwi)
        measured_water_extent = self._get_water_extent(water_mask, bbox)

        return {
            'WATER_LEVEL': measured_water_extent.area / self.nominal_geometry.area,
            'STATUS': water_status,
            'geometry': measured_water_extent
        }

    def _get_water_mask_from_s2(self, ndwi):
        """ Make water detection on input NDWI single band image
        """
        # default threshold (no water detected)
        otsu_thr = 1.0
        status = 0

        # transform NDWI values to [0,1]
        ndwi_std = (ndwi - np.min(ndwi)) / np.ptp(ndwi)

        if len(np.unique(ndwi)) > 1:
            edges = skimage.feature.canny(ndwi_std, sigma=self.canny_sigma, high_threshold=self.canny_threshold)
            edges = skimage.morphology.binary_dilation(edges, self.dilation_disk)
            ndwi_masked = ma.masked_array(ndwi, mask=np.logical_not(edges))

            if len(np.unique(ndwi_masked.data[~ndwi_masked.mask])) > 1:
                # threshold determined using dilated canny edge + otsu
                otsu_thr = skimage.filters.threshold_otsu(ndwi_masked.data[~ndwi_masked.mask])
                status = 1

                # if majority of pixels above threshold have negative NDWI values
                # change the threshold to 0.0
                fraction = np.count_nonzero(ndwi > 0) / np.count_nonzero(ndwi > otsu_thr)
                if fraction < 0.9:
                    otsu_thr = 0.0
                    status = 3
            else:
                # theshold determined with otsu on entire image
                otsu_thr = skimage.filters.threshold_otsu(ndwi)
                status = 2

                # if majority of pixels above threshold have negative NDWI values
                # change the threshold to 0.0
                fraction = np.count_nonzero(ndwi > 0) / np.count_nonzero(ndwi > otsu_thr)
                if fraction < 0.9:
                    otsu_thr = 0.0
                    status = 4

        return (ndwi > otsu_thr).astype(np.uint8), status

    def _get_water_extent(self, water_mask, bbox):
        """
        Returns the polygon of measured water extent.
        """
        src_transform = rasterio.transform.from_bounds(*bbox.lower_left, *bbox.upper_right,
                                                       width=water_mask.shape[1], height=water_mask.shape[0])

        # do vectorization of raster mask
        results = [{'properties': {'raster_val': value}, 'geometry': geo}
                   for geo, value in rasterio.features.shapes(water_mask, transform=src_transform) if value == 1]

        if len(results) == 0:
            return Point(0, 0), 0, 0

        gpd_polygonized_raster = gpd.GeoDataFrame.from_features(results)
        intersection_indices = gpd_polygonized_raster.index[gpd_polygonized_raster.intersects(self.nominal_geometry)]

        measured_water_extent = gpd_polygonized_raster.loc[intersection_indices].cascaded_union
        measured_water_extent = measured_water_extent.buffer(0)

        if self.simplify:
            measured_water_extent = self._simplify_geometry(measured_water_extent, 0.0, 0.0001,
                                                            min(100000, len(self.nominal_geometry.wkt) * 100))

        return measured_water_extent

    @staticmethod
    def _simplify_geometry(geometry, simpl_fact=0.0, simpl_step=0.0001, threshold=20000):
        """
        Simplifies a geometry by reducing the number of vertices.
        """
        while len(geometry.wkt) > threshold:
            geometry = geometry.simplify(simpl_fact, preserve_topology=False)
            simpl_fact += simpl_step

        return geometry

    def execute(self, eopatch):
        """ The execute method
        """
        ndwi_data = eopatch[self.ndwi_feature]

        results = (self._get_water_level(ndwi_data[idx, ..., 0], eopatch.bbox) for idx in range(ndwi_data.shape[0]))

        result_dict = {}
        for wb_result in results:
            for name, value in wb_result.items():
                result_dict[name] = result_dict.get(name, [])
                result_dict[name].append(value)

        gdf = gpd.GeoDataFrame(result_dict, crs={'init': eopatch.bbox.crs.ogc_string()})
        gdf['TIMESTAMP'] = eopatch.timestamp

        eopatch[self.vector_result_feature] = gdf
        eopatch[self.scalar_result_feature] = np.array(result_dict['WATER_LEVEL'])[..., np.newaxis]

        return eopatch

### Definitions of some custom tasks that will be used in the workflow

In [ ]:
class ValidDataPredicate:
    """ An operator to return valid data as an union of pixels with non-zero values and pixels that contain 
    no clouds
    """
    def __call__(self, eopatch):     
        return np.logical_and(eopatch.mask['IS_DATA'].astype(np.bool), 
                              np.logical_not(eopatch.mask['CLM'].astype(np.bool)))


class AddValidDataCoverage(EOTask):
    """ A task that calculates and adds the valid coverage scalar to the EOPatch
    """
    @staticmethod
    def get_coverage(data):
        """ Calculate fraction of pixels with non-zero values
        """
        return 1.0 - np.count_nonzero(data)/np.size(data)
    
    def execute(self, eopatch):

        valid_mask = eopatch.mask['VALID_DATA']
        new_shape = valid_mask.shape[0], valid_mask.shape[1] * valid_mask.shape[2]
        
        coverage = np.apply_along_axis(self.get_coverage, 1, np.reshape(valid_mask, new_shape))
        
        eopatch.scalar['COVERAGE'] = coverage[:, np.newaxis]
        return eopatch

### Initializations of EOTasks

In [ ]:
# A task for downloading RGB bands
# `TRUE-COLOR-S2-L1C` is the name of the layer defined in the Sentinel Hub configurator.
# the arguments are the resolution of the image, max cloud coverage of the whole Satellite tile, 
# and the instance ID for your Sentinel Hub account
input_task = S2L1CWCSInput(layer='TRUE-COLOR-S2-L1C', 
                           resx='20m', resy='20m', maxcc=0.5, 
                           instance_id=INSTANCE_ID)

# A task for downloading the NDWI band combination
# other parameters are copied from the previous task
add_ndwi = S2L1CWCSInput('NDWI', instance_id=INSTANCE_ID)

# A task for cloud detection
# cloud probability map (CLP) and cloud mask (CLM) are calculated at 160 m resolution in order to speed up the process
cloud_classifier = get_s2_pixel_cloud_detector(average_over=2, dilation_size=1, all_bands=False)
cloud_det = AddCloudMaskTask(cloud_classifier, 'BANDS-S2CLOUDLESS', cm_size_y='160m', cm_size_x='160m', 
                             cmask_feature='CLM', cprobs_feature='CLP', instance_id=None)

# A task for adding a raster mask of the nominal water extent (NOMINAL_WATER) 
# raster shape is provided by an existing feature inside of the EOPatch
add_nominal_water = VectorToRaster(gpd.GeoDataFrame(crs={'init': 'epsg:4326'}, geometry=[nominal_geometry]),
                                   (FeatureType.MASK_TIMELESS, 'NOMINAL_WATER'),
                                   values=1, 
                                   raster_shape=(FeatureType.MASK, 'IS_DATA'), 
                                   raster_dtype=np.uint8)

# A task for adding valid data mask to the EOPatch (mask type)
add_valmask = AddValidDataMaskTask(predicate=ValidDataPredicate())

# A task for adding valid data coverage to the EOPatch (scalar type)
add_coverage = AddValidDataCoverage()

# A task for water detection
water_det = WaterDetectionTask((FeatureType.DATA, 'NDWI'),
                               (FeatureType.VECTOR, 'WATER_OUTLINE'),
                               (FeatureType.SCALAR, 'WATER_LEVEL'),
                               nominal_geometry)

### Define and run the workflow

In [ ]:
workflow = LinearWorkflow(
    input_task, 
    add_ndwi, 
    cloud_det, 
    add_nominal_water, 
    add_valmask, 
    add_coverage,
    water_det
)

workflow.dependency_graph()

Let's run the workflow. It will take a few minutes.

In [ ]:
%%time

# time interval definition
TIME_INTERVAL = '2017-01-01', '2019-01-01'

# execute the workflow
result = workflow.execute({
    input_task: {
        'bbox': wb_bbox,
        'time_interval': TIME_INTERVAL
    },
})

### Structure of the `EOPatch`

In [ ]:
# result is in the form of a dictionary
eopatch = result.eopatch()
eopatch

Let's now visualize the first few true-color images of the selected waterbody in the given time series. We see below that some images contain clouds, which causes problems in proper water level detection.

### Plot the NDWI to see how the water detector traces the waterbody outline

In [ ]:
# get aspect ratio of image for better plotting
image_ratio = eopatch.mask_timeless['NOMINAL_WATER'].shape[0] / eopatch.mask_timeless['NOMINAL_WATER'].shape[1]

# plot the NDWI at different dates
fig = plt.figure(figsize=(20, 15 * image_ratio))

for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    ax.imshow(eopatch.data['NDWI'][i].squeeze(), vmin=0, vmax=1)
    ax.axis('off')
    
plt.tight_layout()

### Plot true-color images with the detected water outlines

In [ ]:
def plot_waterbody(image, nominal_geometry, bbox, water_extent,
                   color_nominal='white', color_current='xkcd:lime', ax=None, lw=2):
    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

    ax.imshow(image, extent=[bbox.min_x, bbox.max_x, bbox.min_y, bbox.max_y])

    draw_geometry(ax, nominal_geometry, color_nominal, lw=lw)
    draw_geometry(ax, water_extent, color_current, lw=lw)


def draw_geometry(ax, geometry, color, lw=2):
    if isinstance(geometry, Polygon):
        draw_polygon(ax, geometry, color, lw=lw)
    elif isinstance(geometry, MultiPolygon):
        for polygon in geometry:
            draw_polygon(ax, polygon, color, lw=lw)


def draw_polygon(ax, polygon, color, lw=2):
    if polygon is None or polygon.exterior is None:
        return None

    x, y = polygon.exterior.coords.xy
    xy = np.moveaxis(np.array([x, y]), 0, -1)
    patch = ax.add_patch(matplotlib.patches.Polygon(xy, closed=True, edgecolor=color, fill=False, lw=lw))
    
    draw_outline(patch, 4)


def draw_outline(o, lw):
    o.set_path_effects([matplotlib.patheffects.Stroke(linewidth=lw, foreground='black'),
                        matplotlib.patheffects.Normal()])

In [ ]:
image_ratio = eopatch.data['TRUE-COLOR-S2-L1C'][0].shape[0] / eopatch.data['TRUE-COLOR-S2-L1C'][0].shape[1]

fig = plt.figure(figsize=(20, 15 * image_ratio))

for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    plot_waterbody(eopatch.data['TRUE-COLOR-S2-L1C'][i], nominal_geometry,
                   wb_bbox, eopatch.vector['WATER_OUTLINE']['geometry'][i], ax=ax)
    ax.axis('off')
    
plt.tight_layout(pad=0)

### Plotting the detected water levels

In [ ]:
# This is just to suppress a plotting warning
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

In [ ]:
def plot_water_levels(eopatch, max_coverage=1.0):
    fig, ax = plt.subplots(figsize=(20, 7))

    dates = np.array(eopatch.timestamp)
    ax.plot(dates[eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            eopatch.scalar['WATER_LEVEL'][eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            'bo-', alpha=0.7, label='Water Level')
    ax.plot(dates[eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            eopatch.scalar['COVERAGE'][eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            '--', color='gray', alpha=0.7, label='Cloud Coverage')
    ax.set_ylim(0.0,1.1)
    ax.set_xlabel('Date')
    ax.set_ylabel('Water Level')
    ax.set_title('Detected Water Level')
    ax.grid(axis='y')
    ax.legend(loc='best')
    return ax

# plot the water level with no cloudy scene filtering (accept all clouds)
ax = plot_water_levels(eopatch, 1.0);

We see a lot of fluctuations in the water level. At the same time, cloud coverage is plotted, and it shares the same dates as the water level outliers.

Let us now set a threshold for the maximum cloud coverage of 2% and filter out the dates which correspond to cloudy scenes. This is done by filtering out the dates which have a value of `eopatch.scalar['COVERAGE']` larger than 0.02.

In [ ]:
plot_water_levels(eopatch, 0.02);